In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16879425
paper_name = 'fujita_iwahashi_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['Table1.txt','Table2.txt','Table3.txt']

In [11]:
original_data_list = []
for f in files:
    original_data = pd.read_csv('raw_data/' + f, sep='\t', skiprows=1)
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['orf'] = original_data['ORF'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    # Make sure everything translated ok
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    original_data['data'] = -1
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 138 x 8
              #  ORF Gene name Description  Unnamed: 4  Unnamed: 5  \
index_input                                                          
137         NaN  NaN       NaN         NaN         NaN         NaN   

             Unnamed: 6  Unnamed: 7  orf  
index_input                               
137                 NaN         NaN  NAN  
(137, 1)
Original data dimensions: 122 x 8
Empty DataFrame
Columns: [#, ORF, Gene name, Description, Unnamed: 4, Unnamed: 5, Unnamed: 6, Unnamed: 7, orf]
Index: []
(122, 1)
Original data dimensions: 48 x 8
Empty DataFrame
Columns: [#, ORF, Gene name, Description, Unnamed: 4, Unnamed: 5, Unnamed: 6, Unnamed: 7, orf]
Index: []
(48, 1)


In [14]:
original_data = pd.concat(original_data_list, axis=1)

In [16]:
original_data.index.name='orf'

In [17]:
original_data[original_data.isnull()] = 0

In [18]:
original_data.head()

,data,data,data
orf,,,
YBL006C,-1.0,-1.0,0.0
YBL058W,-1.0,-1.0,-1.0
YBR029C,-1.0,-1.0,0.0
YBR036C,-1.0,0.0,-1.0
YBR127C,-1.0,-1.0,-1.0


In [19]:
original_data.shape

(166, 3)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [497, 5004, 5005]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,497,5004,5005
data_type,value,value,value
orf,,,
YBL006C,-1.0,-1.0,0.0
YBL058W,-1.0,-1.0,-1.0
YBR029C,-1.0,-1.0,0.0
YBR036C,-1.0,0.0,-1.0
YBR127C,-1.0,-1.0,-1.0


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,497,5004,5005
,data_type,value,value,value
gene_id,orf,,,
94,YBL006C,-1.0,-1.0,0.0
146,YBL058W,-1.0,-1.0,-1.0
225,YBR029C,-1.0,-1.0,0.0
232,YBR036C,-1.0,0.0,-1.0
321,YBR127C,-1.0,-1.0,-1.0


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,497,5004,5005,497,5004,5005
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
94,YBL006C,-1.0,-1.0,0.0,-5.771817,-6.116358,0.000000
146,YBL058W,-1.0,-1.0,-1.0,-5.771817,-6.116358,-9.751068
225,YBR029C,-1.0,-1.0,0.0,-5.771817,-6.116358,0.000000
232,YBR036C,-1.0,0.0,-1.0,-5.771817,0.000000,-9.751068
321,YBR127C,-1.0,-1.0,-1.0,-5.771817,-6.116358,-9.751068


# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 16879425...
Inserting the new data...


100%|██████████| 3/3 [00:01<00:00,  2.43it/s]

Updating the data_modified_on field...
